# 自定义 LLM 代理

本文档将介绍如何创建自定义 LLM 代理。

LLM 代理分为三个部分：

- PromptTemplate: 这是一个提示模板，用于指导语言模型的操作
- LLM: 这是为代理提供动力的语言模型
- `stop` sequence: 一旦发现该字符串，即指示 LLM 停止生成
- OutputParser: 这决定了如何解析 LLMOutput 为 AgentAction 或 AgentFinish 对象


LLMAgent 用于 AgentExecutor 中。该 AgentExecutor 主要可以被视为一个循环：
1. 将用户输入和所有先前步骤传递给代理（在本例中为 LLMAgent）
2. 如果代理返回一个 `AgentFinish`，则直接返回给用户
3. 如果代理返回一个 `AgentAction`，则使用它来调用工具并获取一个 `Observation`
4. 重复以上步骤，将 `AgentAction` 和 `Observation` 传递回代理，直到发出一个 `AgentFinish`
    
“AgentAction” 是一个响应，它由 “action” 和 “action_input” 组成。“action” 指要使用哪个工具，“action_input” 指要使用的输入。还可以提供 `log` 作为更多的上下文（可用于日志记录、跟踪等）。

“AgentFinish” 是一个响应，包含最终要发送回用户的消息。这应该用于结束代理运行。
        
在本文档中，我们将介绍如何创建自定义 LLM 代理。

## 设置环境

执行必要导入等操作。

In [1]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## 设置工具

设置代理可能想要使用的任何工具。这可能需要放在提示中（以便代理知道要使用这些工具）。

In [2]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

## 提示模板

这指导代理做什么。一般情况下，模板应该包括：
    
- `tools`：代理可以使用哪些工具，以及何时和如何调用它们。
- `intermediate_steps`：这些是先前（“AgentAction”，“Observation”）对的元组。通常不会直接传递到模型中，但提示模板以特定方式格式化它们。
- `input`：通用用户输入

In [3]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [4]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [5]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## 输出解析器

输出解析器负责将 LLM 输出解析为“AgentAction”和“AgentFinish”。这通常高度依赖于使用的提示。

这是您可以更改解析以进行重试、处理空格等操作的位置

In [6]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [7]:
output_parser = CustomOutputParser()

## 设置 LLM

选择要使用的 LLM！

In [8]:
llm = OpenAI(temperature=0)

## 定义停止序列

这很重要，因为它告诉 LLM 何时停止生成。

这严重取决于您使用的提示和模型。通常，您希望这是您在提示中用于表示“Observation”开头的令牌（否则，LLM 可能会为您产生幻觉）。

## 设置代理

现在我们可以将所有内容组合起来设置代理

In [9]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

## 使用代理

现在我们可以使用它了！

In [11]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [12]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada in 2023
Action: Search
Action Input: Population of Canada in 2023

Observation:The current population of Canada is 38,658,314 as of Wednesday, April 12, 2023, based on Worldometer elaboration of the latest United Nations data. I now know the final answer
Final Answer: Arrr, there be 38,658,314 people livin' in Canada as of 2023!

> Finished chain.


"Arrr, there be 38,658,314 people livin' in Canada as of 2023!"

## 添加记忆

如果您要将记忆添加到代理中，您需要：

1. 在自定义提示中为 chat_history 添加一个位置
2. 为代理执行器添加内存对象。

In [29]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [30]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [31]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [32]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [33]:
from langchain.memory import ConversationBufferWindowMemory

In [42]:
memory=ConversationBufferWindowMemory(k=2)

In [43]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [44]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada in 2023
Action: Search
Action Input: Population of Canada in 2023

Observation:The current population of Canada is 38,658,314 as of Wednesday, April 12, 2023, based on Worldometer elaboration of the latest United Nations data. I now know the final answer
Final Answer: Arrr, there be 38,658,314 people livin' in Canada as of 2023!

> Finished chain.


"Arrr, there be 38,658,314 people livin' in Canada as of 2023!"

In [45]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...
Thought: I need to find out how many people live in Mexico.
Action: Search
Action Input: How many people live in Mexico as of 2023?

Observation:The current population of Mexico is 132,679,922 as of Tuesday, April 11, 2023, based on Worldometer elaboration of the latest United Nations data. Mexico 2020 ... I now know the final answer.
Final Answer: Arrr, there be 132,679,922 people livin' in Mexico as of 2023!

> Finished chain.


"Arrr, there be 132,679,922 people livin' in Mexico as of 2023!"